## Guided project, Answering Business Questions On The Chinook database

#### Connect our Jupyter Notebook to our database file.

In [2]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

#### Let's start by getting familiar with our data. 

In [3]:
%%sql

SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [4]:
%%sql

select * from album limit 5;

Done.


album_id,title,artist_id
1,For Those About To Rock We Salute You,1
2,Balls to the Wall,2
3,Restless and Wild,2
4,Let There Be Rock,1
5,Big Ones,3


In [5]:
%%sql

pragma table_info('album');

Done.


cid,name,type,notnull,dflt_value,pk
0,album_id,INTEGER,1,None,1
1,title,NVARCHAR(160),1,None,0
2,artist_id,INTEGER,1,None,0


In [6]:
%%sql

pragma table_info('customer');

Done.


cid,name,type,notnull,dflt_value,pk
0,customer_id,INTEGER,1,None,1
1,first_name,NVARCHAR(40),1,None,0
2,last_name,NVARCHAR(20),1,None,0
3,company,NVARCHAR(80),0,None,0
4,address,NVARCHAR(70),0,None,0
5,city,NVARCHAR(40),0,None,0
6,state,NVARCHAR(40),0,None,0
7,country,NVARCHAR(40),0,None,0
8,postal_code,NVARCHAR(10),0,None,0
9,phone,NVARCHAR(24),0,None,0


In [7]:
%%sql

pragma table_info('media_type');

Done.


cid,name,type,notnull,dflt_value,pk
0,media_type_id,INTEGER,1,None,1
1,name,NVARCHAR(120),0,None,0


#### Analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

Total dollar amount of sales assigned to each sales support agent within the company.

In [8]:
%%sql

select min(hire_date), max(hire_date) from employee

Done.


min(hire_date),max(hire_date)
2016-05-01 00:00:00,2017-10-17 00:00:00


In [9]:
%%sql

select min(invoice_date), max(invoice_date) from invoice

Done.


min(invoice_date),max(invoice_date)
2017-01-03 00:00:00,2020-12-30 00:00:00


In [10]:
%%sql


select 
e.*,
--e.employee_id,
--e.first_name || ' ' || e.last_name employee_name,
--e.title,
sum(i.total) employee_sales,
sum(i.total)/((select max(invoice_date) from invoice)-e.hire_date)  employee_performance

from employee e
inner join customer c on e.employee_id = c.support_rep_id
inner join invoice i on i.customer_id = c.customer_id
--where e.title = 'Sales Support Agent'
group by e.employee_id
order by employee_sales desc
;

Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email,employee_sales,employee_performance
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com,1731.5100000000039,577.1700000000013
4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2017-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com,1584.0000000000034,528.0000000000011
5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2017-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com,1393.920000000002,464.6400000000006


There are no significant differences between the employees, exept we noticed that the top saler is the yougest. We suppose that the sells have a uniforme distribution in time.

#### Returns each genre, with the number of tracks sold in the USA:
- in absolute numbers
- in percentages.

In [11]:
%%sql

drop view  if exists tracks_sold_usa;

CREATE VIEW tracks_sold_usa AS


with tracks_sold_usa as(
select g.name, count(t.track_id) track_sold_usa

from customer c
inner join invoice i on i.customer_id = c.customer_id
inner join invoice_line il on il.invoice_id = i.invoice_id
inner join track t on t.track_id = il.track_id
inner join genre g on g.genre_id = t.genre_id
where c.country = 'USA'
group by g.name
order by track_sold_usa desc
)
select name, track_sold_usa, 
track_sold_usa * 100.0 / (select sum(track_sold_usa) from tracks_sold_usa) perc_track_sold_usa
from tracks_sold_usa tsu
group by name
order by track_sold_usa desc;

select * from tracks_sold_usa;






Done.
Done.
Done.


name,track_sold_usa,perc_track_sold_usa
Rock,561,53.37773549000951
Alternative & Punk,130,12.369172216936251
Metal,124,11.798287345385347
R&B/Soul,53,5.042816365366318
Blues,36,3.4253092293054235
Alternative,35,3.330161750713606
Latin,22,2.093244529019981
Pop,22,2.093244529019981
Hip Hop/Rap,20,1.9029495718363463
Jazz,14,1.3320647002854424


In [12]:
%%sql

select * from tracks_sold_usa 
where name in ('Hip Hop/Rap','Pop','Blues','Alternative & Punk')


Done.


name,track_sold_usa,perc_track_sold_usa
Alternative & Punk,130,12.369172216936251
Blues,36,3.4253092293054235
Pop,22,2.093244529019981
Hip Hop/Rap,20,1.9029495718363463


In [13]:
%%sql
drop view  if exists tracks_sold_usa;

Done.


[]

Having to pick three from the list :

- Artist Name	/ Genre
 - Regal	/ Hip-Hop
 - Red Tone	/ Punk
 - Meteor and the Girls	/ Pop
 - Slim Jim Bites	/ Blues
 
We might want to leave Regal aside because we have the lowest sale for the Hip Hop/Rap genre.
Wich means less market or less experience to deal with this artist.

 #### Analyze the sales data for customers from each different country.

In [14]:
%%sql

select 
case 
    when 
    (select count(distinct cu.cutomer_id) 
     from customer cu
     where cu.country = c.country     
     ) =1
    then 'Other'
    else c.country
end as country,

from customer c
;



(sqlite3.OperationalError) near "from": syntax error
[SQL: select 
case 
    when 
    (select count(distinct cu.cutomer_id) 
     from customer cu
     where cu.country = c.country     
     ) =1
    then 'Other'
    else c.country
end as country,

from customer c
;]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


In [15]:
%%sql

drop view  if exists country_customer;
CREATE VIEW country_customer AS
select c.country, count(c.customer_id) as customers_per_country
from customer c
group by c.country
--having customers_per_country = 1
order by customers_per_country desc;

select * from country_customer;


Done.
Done.
Done.


country,customers_per_country
USA,13
Canada,8
Brazil,5
France,5
Germany,4
United Kingdom,3
Czech Republic,2
India,2
Portugal,2
Argentina,1


In [81]:
%%sql


    
with country_other as(
select
--c.country,
case when
(select 
count(distinct cc.customer_id)
from customer cc
where cc.country = c.country) =1 
then 'Other'
else c.country
end as country,
c.customer_id,
i.total,
i.invoice_id
from customer c
inner join invoice i on i.customer_id = c.customer_id
)
select 
co.country,
count(distinct co.customer_id) number_of_customers,
round(sum(co.total),2) value_of_sales,
round(sum(co.total)/count(distinct co.customer_id),2) average_value_of_sales_per_costumer,
round(sum(co.total)/count(distinct co.invoice_id),2) average_order_value,
case when co.country = 'Other' then 1 else 0 end as sort
from country_other co
group by co.country
order by sort asc, number_of_customers desc



Done.


country,number_of_customers,value_of_sales,average_value_of_sales_per_costumer,average_order_value,sort
USA,13,1040.49,80.04,7.94,0
Canada,8,535.59,66.95,7.05,0
Brazil,5,427.68,85.54,7.01,0
France,5,389.07,77.81,7.78,0
Germany,4,334.62,83.66,8.16,0
United Kingdom,3,245.52,81.84,8.77,0
Czech Republic,2,273.24,136.62,9.11,0
India,2,183.15,91.57,8.72,0
Portugal,2,185.13,92.57,6.38,0
Other,15,1094.94,73.0,7.45,1


There are countries with few customers and a high average value of sale. So it might be interesting to start advertising compagns in :
- Czech Republic
- India
- Portugal

But because of the few data samples it migh be wise to start with small compagns and analyse the results befaore going further.

#### considering to purchase only the most popular tracks from each album from record companies, or of purchasing every track from an album.

In this instance, we have two edge cases to consider:

- Albums that have only one or two tracks are likely to be purchased by customers as part of a collection of individual tracks.


- Customers may decide to manually select every track from an album, and then add a few individual tracks from other albums to their purchase.

In the first case, since our analysis is concerned with maximizing revenue we can safely ignore albums consisting of only a few tracks. The company has previously done analysis to confirm that the second case does not happen often, so we can ignore this case also.

Let's test the use of the suggested method with EXCEPT and AND to check if two selections are the same.

In [145]:
%%sql

select(
((select title from album where album_id in (1,2)
except select title from album where album_id in (1,2)) is null)
and
((select title from album where album_id in (1,2)
except select title from album where album_id in (1,2)) is null) 
) is_same

Done.


is_same
1


In [139]:
%%sql

with invoice_album as (
select distinct 
i.invoice_id, a.album_id,  a.title, t.track_id
from
invoice i
inner join invoice_line il on il.invoice_id = i.invoice_id 
inner join track t on t.track_id = il.track_id
inner join album a on a.album_id = t.album_id
),
album_purchase as (
select 
ia.invoice_id,

((select track_id from track where album_id = ia.album_id 
except
select track_id from invoice_album where invoice_id = ia.invoice_id) is null)
and
((select track_id from invoice_album where invoice_id = ia.invoice_id 
except
select track_id from track where album_id = ia.album_id) is null)

as is_album_purchase

from invoice_album ia

--where is_album_purchase = 1
group by ia.invoice_id
)
select round(sum(is_album_purchase) * 100.00 /count(invoice_id),2) albums_purchase_percent,
100 - round(sum(is_album_purchase) * 100.00 /count(invoice_id),2) tracks_purchase_percent
from album_purchase

;


Done.


albums_purchase_percent,tracks_purchase_percent
18.57,81.43


The majority of sales are tracks purchases. but almost 20% of the sales are album purchases. Depending on the savings of the new strategy, we might test gradualy and being very cotious the strategy of replacing albums with their most popular tracks.